In [113]:
from skimage.io import imread
import pandas as pd
import numpy as np
import torch
import os

In [114]:
import ultralytics

In [115]:
image_dir = 'cards_dataset/images/'

In [116]:
data_test = []
data_train = []
test_labels = pd.read_csv(image_dir+'test_labels.csv')
train_labels = pd.read_csv(image_dir+'train_labels.csv')

In [117]:
train_labels

,filename,width,height,class,xmin,ymin,xmax,ymax
0,cam_image1.jpg,480,270,queen,173,24,260,137
1,cam_image1.jpg,480,270,queen,165,135,253,251
2,cam_image1.jpg,480,270,ten,255,96,337,208
3,cam_image10.jpg,960,540,ten,501,116,700,353
4,cam_image10.jpg,960,540,queen,261,124,453,370
...,...,...,...,...,...,...,...,...
522,IMG_2700.JPG,378,504,ace,74,105,303,425
523,IMG_2701.JPG,378,504,jack,64,102,313,453
524,IMG_2702.JPG,378,504,nine,85,133,290,419
525,IMG_2703.JPG,378,504,ace,82,152,277,422


In [118]:
classes = train_labels['class'].unique()
classes

array(['queen', 'ten', 'nine', 'king', 'jack', 'ace'], dtype=object)

Преобразуем в YOLO формат

In [119]:
test_labels['x_center'] = ((test_labels['xmin']+test_labels['xmax'])/2) / (test_labels['xmax']-test_labels['xmin'])
test_labels['y_center'] = ((test_labels['ymin']+test_labels['ymax'])/2) / (test_labels['ymax']-test_labels['ymin'])
test_labels['bbox_width'] = test_labels['width'] / (test_labels['xmax']-test_labels['xmin'])
test_labels['bbox_height'] = test_labels['height'] / (test_labels['ymax']-test_labels['ymin'])

In [120]:
test_labels

,filename,width,height,class,xmin,ymin,xmax,ymax,x_center,y_center,bbox_width,bbox_height
0,cam_image2.jpg,960,540,king,312,30,485,249,2.303468,0.636986,5.549133,2.465753
1,cam_image2.jpg,960,540,king,514,24,694,245,3.355556,0.608597,5.333333,2.443439
2,cam_image2.jpg,960,540,king,305,263,489,519,2.157609,1.527344,5.217391,2.109375
3,cam_image2.jpg,960,540,king,515,267,704,523,3.224868,1.542969,5.079365,2.109375
4,cam_image4.jpg,960,540,jack,297,18,459,237,2.333333,0.582192,5.925926,2.465753
...,...,...,...,...,...,...,...,...,...,...,...,...
138,IMG_2677.JPG,378,504,ten,80,114,163,357,1.463855,0.969136,4.554217,2.074074
139,IMG_2678.JPG,378,504,jack,119,228,275,445,1.262821,1.550691,2.423077,2.322581
140,IMG_2678.JPG,378,504,jack,42,239,132,459,0.966667,1.586364,4.200000,2.290909
141,IMG_2686.JPG,378,504,queen,143,150,319,378,1.312500,1.157895,2.147727,2.210526


In [121]:
train_labels['x_center'] = ((train_labels['xmin']+train_labels['xmax'])/2) / (train_labels['xmax']-train_labels['xmin'])
train_labels['y_center'] = ((train_labels['ymin']+train_labels['ymax'])/2) / (train_labels['ymax']-train_labels['ymin'])
train_labels['bbox_width'] = train_labels['width'] / (train_labels['xmax']-train_labels['xmin'])
train_labels['bbox_height'] = train_labels['height'] / (train_labels['ymax']-train_labels['ymin'])

In [122]:
train_labels

,filename,width,height,class,xmin,ymin,xmax,ymax,x_center,y_center,bbox_width,bbox_height
0,cam_image1.jpg,480,270,queen,173,24,260,137,2.488506,0.712389,5.517241,2.389381
1,cam_image1.jpg,480,270,queen,165,135,253,251,2.375000,1.663793,5.454545,2.327586
2,cam_image1.jpg,480,270,ten,255,96,337,208,3.609756,1.357143,5.853659,2.410714
3,cam_image10.jpg,960,540,ten,501,116,700,353,3.017588,0.989451,4.824121,2.278481
4,cam_image10.jpg,960,540,queen,261,124,453,370,1.859375,1.004065,5.000000,2.195122
...,...,...,...,...,...,...,...,...,...,...,...,...
522,IMG_2700.JPG,378,504,ace,74,105,303,425,0.823144,0.828125,1.650655,1.575000
523,IMG_2701.JPG,378,504,jack,64,102,313,453,0.757028,0.790598,1.518072,1.435897
524,IMG_2702.JPG,378,504,nine,85,133,290,419,0.914634,0.965035,1.843902,1.762238
525,IMG_2703.JPG,378,504,ace,82,152,277,422,0.920513,1.062963,1.938462,1.866667


Формат: [class x_center y_center width height]

In [123]:
cols = [
 'class',
 'x_center',
 'y_center',
 'bbox_width',
 'bbox_height']

In [124]:
def pandas_to_dict(labels):
    res_dict = {}
    for key in labels['filename'].unique().tolist():
        res_dict[key] = []
    for i in range(len(labels)):
        filename = labels.iloc[i,:]['filename']
        res_dict[filename].append(labels.iloc[i,:][cols].to_list())
    return res_dict


In [125]:
labels_train_YOLO = pandas_to_dict(train_labels)
labels_test_YOLO = pandas_to_dict(test_labels)

In [126]:
labels_train_YOLO


{'cam_image2.jpg': [['king',
   2.3034682080924855,
   0.636986301369863,
   5.5491329479768785,
   2.4657534246575343],
  ['king',
   3.3555555555555556,
   0.6085972850678733,
   5.333333333333333,
   2.4434389140271495],
  ['king', 2.157608695652174, 1.52734375, 5.217391304347826, 2.109375],
  ['king', 3.2248677248677247, 1.54296875, 5.079365079365079, 2.109375]],
 'cam_image4.jpg': [['jack',
   2.3333333333333335,
   0.5821917808219178,
   5.925925925925926,
   2.4657534246575343],
  ['jack',
   3.1032608695652173,
   0.5401785714285714,
   5.217391304347826,
   2.4107142857142856],
  ['jack',
   2.1882352941176473,
   1.6020408163265305,
   5.647058823529412,
   2.204081632653061],
  ['jack',
   3.2111111111111112,
   1.5647773279352226,
   5.333333333333333,
   2.1862348178137654]],
 'cam_image45.jpg': [['jack',
   5.11875,
   1.2004219409282701,
   6.0,
   2.278481012658228],
  ['nine',
   3.7865497076023393,
   1.0991735537190082,
   5.614035087719298,
   2.231404958677686],
  

Для каждой картинки создадим текстовый файл с разметкой

In [152]:
def create_txt_labels(dict_labels ,dir = None):
    for key in dict_labels.keys():
        filename = key.split('.')[0]
        with open(dir + filename + '.txt', mode='w') as curr:
            for arr_of_labels in dict_labels[key]:
                for item in arr_of_labels:
                    curr.write(str(item)+' ')
                    
                curr.write('\n')

In [153]:
DIR_TRAIN = 'cards_dataset/labels/train/'
create_txt_labels(train_labels_YOLO, DIR_TRAIN)

In [155]:
DIR_TEST = 'cards_dataset/labels/val/'
create_txt_labels(test_labels_YOLO, DIR_TEST)